<a href="https://colab.research.google.com/github/Shah12345678901/Classification-EMDS-/blob/main/MobileNetv2_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Libraries**

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import os
import tensorflow as tf

**Data Loading**

In [ ]:
PATH='/content/drive/MyDrive/Colab Notebooks/EMDS5-Original'


train_dir = os.path.join(PATH, 'Train')
validation_dir = os.path.join(PATH, 'Val')



BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

Found 672 files belonging to 21 classes.


In [ ]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

Found 168 files belonging to 21 classes.


**Data Prepration **

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [ ]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

Number of validation batches: 5
Number of test batches: 1


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

**Data Agumentation**

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

**Tranfer Learning**

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 5, 5, 1280)


In [ ]:
base_model.trainable = False

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 80, 80, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 80, 80, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 80, 80, 32)   0           ['bn_Conv1[0][

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)



(32, 1280)


In [ ]:
prediction_layer = tf.keras.layers.Dense(21)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 21)


In [ ]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 160, 160, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 160, 160, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 160, 160, 3)      0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_160 (Funct  (None, 5, 5, 1280)       2257984   
 ional)                                                          
                                                             

In [ ]:
len(model.trainable_variables)

2

In [ ]:
initial_epochs = 25

loss0, accuracy0 = model.evaluate(validation_dataset)

5/5 [==============================] - 3s 80ms/step - loss: 9.5878 - accuracy: 0.0588


In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: 9.59
initial accuracy: 0.06


In [ ]:
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

Epoch 1/25
21/21 [==============================] - 9s 216ms/step - loss: 11.8932 - accuracy: 0.0446 - val_loss: 10.6652 - val_accuracy: 0.0735
Epoch 2/25
21/21 [==============================] - 6s 216ms/step - loss: 11.8661 - accuracy: 0.0446 - val_loss: 10.9832 - val_accuracy: 0.1029
Epoch 3/25
21/21 [==============================] - 6s 218ms/step - loss: 11.6692 - accuracy: 0.0729 - val_loss: 11.2571 - val_accuracy: 0.0956
Epoch 4/25
21/21 [==============================] - 6s 217ms/step - loss: 11.8433 - accuracy: 0.0833 - val_loss: 11.3084 - val_accuracy: 0.1324
Epoch 5/25
21/21 [==============================] - 7s 272ms/step - loss: 11.6996 - accuracy: 0.0952 - val_loss: 11.1540 - val_accuracy: 0.1324
Epoch 6/25
21/21 [==============================] - 6s 216ms/step - loss: 11.6292 - accuracy: 0.1042 - val_loss: 10.6059 - val_accuracy: 0.1176
Epoch 7/25
21/21 [==============================] - 6s 215ms/step - loss: 11.0443 - accuracy: 0.1250 - val_loss: 8.1579 - val_accuracy: 

**Fine Tunining**

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

Number of layers in the base model:  154


In [ ]:
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 160, 160, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 160, 160, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 160, 160, 3)      0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_160 (Funct  (None, 5, 5, 1280)       2257984   
 ional)                                                          
                                                             

In [ ]:
len(model.trainable_variables)

56

In [ ]:
fine_tune_epochs = 25
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

Epoch 25/50
21/21 [==============================] - 11s 251ms/step - loss: 3.1652 - accuracy: 0.2039 - val_loss: 3.0889 - val_accuracy: 0.2206
Epoch 26/50
21/21 [==============================] - 6s 228ms/step - loss: 3.0581 - accuracy: 0.1935 - val_loss: 3.1093 - val_accuracy: 0.2353
Epoch 27/50
21/21 [==============================] - 6s 227ms/step - loss: 3.0355 - accuracy: 0.1994 - val_loss: 3.0445 - val_accuracy: 0.2059
Epoch 28/50
21/21 [==============================] - 6s 226ms/step - loss: 3.0522 - accuracy: 0.1890 - val_loss: 3.0445 - val_accuracy: 0.1544
Epoch 29/50
21/21 [==============================] - 6s 225ms/step - loss: 3.0317 - accuracy: 0.1979 - val_loss: 3.0445 - val_accuracy: 0.1838
Epoch 30/50
21/21 [==============================] - 6s 226ms/step - loss: 3.0838 - accuracy: 0.1860 - val_loss: 3.0445 - val_accuracy: 0.2132
Epoch 31/50
21/21 [==============================] - 6s 227ms/step - loss: 3.0584 - accuracy: 0.1830 - val_loss: 3.0445 - val_accuracy: 0.191

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

1/1 [==============================] - 1s 1s/step - loss: 3.0445 - accuracy: 0.2500
Test accuracy : 0.25


**Here test accuracy is 25%**